In [2]:
import os
import csv
import owlready2

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




'*' indicates mandatory field

'**' indicates at least one attribute in the group required

all other attributes are optional


In [16]:
templates_path = "data/biosamples/templates/"

attributes_dict = {}
required_attributes = set()

for filename in os.listdir(templates_path):
    with open(templates_path + filename, 'r') as f:
        reader = csv.reader(f, delimiter='\t', quotechar='"')
        for row in reader:
            if not row or row[0].startswith("#"):
                continue
            attributes_dict[filename.strip(".tsv")] = [attribute for attribute in row if attribute[0] == "*" and attribute[1] != "*"]

# print(attributes_dict)
# prints the attributes that are common to all biosamples templates (from NCBI packages)
print(set.intersection(*[set(attributes_dict[template]) for template in attributes_dict]))

{'Beta-lactamase.1.0': ['*sample_name', '*organism', '*beta_lactamase_family', '*carbapenemase', '*edta_inhibitor_tested'], 'Human.1.0': ['*sample_name', '*organism', '*isolate', '*age', '*biomaterial_provider', '*sex', '*tissue'], 'Invertebrate.1.0': ['*sample_name', '*organism', '*collection_date', '*geo_loc_name', '*tissue'], 'Metagenome.environmental.1.0': ['*sample_name', '*organism', '*collection_date', '*geo_loc_name', '*lat_lon'], 'Microbe.1.0': ['*sample_name', '*organism', '*collection_date', '*geo_loc_name', '*sample_type'], 'Model.organism.animal.1.0': ['*sample_name', '*organism', '*sex', '*tissue'], 'OneHealthEnteric.1.0': ['*sample_name', '*organism', '*collected_by', '*collection_date', '*geo_loc_name', '*isolation_source', '*purpose_of_sampling', '*source_type', '*strain'], 'Pathogen.cl.1.0': ['*sample_name', '*organism', '*collected_by', '*collection_date', '*geo_loc_name', '*host', '*host_disease', '*isolation_source', '*lat_lon'], 'Pathogen.env.1.0': ['*sample_name'

In [ ]:
onto_path.append("data/ontologies/")